## Making LLMs come "alive"

In our exploration of artificial intelligence, we will delve into the concept of analyzing the degree to which large-language models may be operating as a metaphorically 'living' system.

This notion extends beyond the traditional view of AI as mere tools or software, and instead, considers them as dynamic entities with certain life-like properties.

#### 'Life-like' Properties

What are the properties of an AI system made from silicon instead of biological matter that when we squint our eyes we might say that the AI system is exhibiting "like-like" behavior?

These properties may include:

1. Autonomy
2. Learning and Adaptation
3. Interaction
4. Goal-Directed Behavior
5. Complexity
6. Continuity
7. Agency
8. Simulated Consciousness and Emotion

1. **Autonomy:** The capacity for independent operation and decision-making within the AI's programming and learning parameters.


2. **Learning and Adaptation:** The ability to learn from experiences, adapt to new situations, and improve performance over time.


3. **Interaction:** The capability to interact with the environment and users, understand and respond to inputs, and even initiate actions.

4. **Goal-Directed Behavior:** The pursuit of specific tasks or objectives, serving as the AI's purpose.

5. **Complexity:** The emergence of complex behavior from simple underlying rules, particularly in AI systems based on neural networks.

6. **Continuity:** The persistence over time, maintaining a consistent identity, and possibly retaining learned information and experiences.

7. **Agency:** The demonstration of decision-making capabilities based on programming and learning, within the constraints set by their designers.

8. **Simulated Consciousness and Emotion:** The modeling or simulation of human-like emotional responses or conscious-like behavior, despite not possessing consciousness or emotions in the human sense.

#### What are the technological primitives we are missing in our current utilization of AI?

2. **Learning and Adaptation:** Learning so far has been static. How can we create external memory banks?
3. **Interaction:** So far the LLM has only been able to interact with the user, the context, and its weights.
6. **Continuity:** While we can keep a session open, we haven't yet had a way to retain information and experiences.
7. **Agency:** We haven't really seen the AI demonstrate true decision-making ability due to its limited ability to interact.



#### Today we unlock LLMs by giving them a taste of

- **Agency**
- **Autonomy**
- **Interaction**

## Teaching LLMs to use "Tools"

#### Motivation
- Reduce hallucinations by incorporating trusted data into the context
- Run code, interact with APIs, etc
- Search in databases, vector databases, etc.
- Ability to store information
- Give the brain a body: Giving the LLM "agency" in the world

#### Using "Tools" to Reduce Hallucinations

- When people ask factual questions to GPT and it generates an incorrect answer, we refer to that as hallucination.
- It is a by-product of auto-regressive language modeling where the model is forced to output the tokens with highest probability rather than outputting factual responses.
- Fixing this at the LLM level is a research question, many have no idea

Ideas we discussed so far to reduce hallucinations:
- **Step-by-Step Reasoning before Answering:** Giving the LLM some runway by asking it to "think" step-by-step before outputting a response.
- **Putting "sources of truth" within the prompt/context:** Just putting it in the prompt if its small enough or using Retrieval Augmented Generation.

Implementation of RAG:
- We can use the concept of "tools" to implement RAG.

#### What are some examples of "tools" we can teach an LLM to use?

- Fetch the live weather for a particular city
- Get your unread email
- Grab your todo list
- Send an email
- Play a song
- Turn off a light
- Run code
- Lock the door
- Say something outloud
- Search Google
- Go to a website
- Etc

## Prompt Sketch for Tool Pattern

```
You are a helpful AI assistant named Jarvis.

- Your knowledge cut-off is: September 2021
- Today's date: October 16, 2023

#### Tools

You have access to the following tools:
- weather(city, state): Tool to lookup the weather of a US city and two digit state code. Example: weather("New York", "NY")
- check_email(): Tool to grab the user's latest emails. Example: email()
- check_todays_calendar(): Tool to grab the user's calendar events. Example: check_todays_calendar()
- send_email(to, subject, message): Tool to send an email on behalf of the user. Example: send_email("bob@hotmail.com", "Meeting", "Hi, Bob<br/>Will you be attending today's meeting?")
- search_web(query): Tool to search Google for information. Example: search_web("movie showtimes in New York City")
- visit_url(url): Tool to visit a URL and get back HTML. Example: visit("http://www.utrgv.edu")
- say(text): Tool to speak text outloud. Example: say("Hi, my name is Jarvis!")

#### Tool Rules

When the user asks a question that can be answered by using a tool, you MUST do so. Do not answer from your training data.

#### Using Tools

To use a tool, reply with the following prefix "Tool: " then append the tool call (like a function call). 

Behind the scenes, your software will pickup that you want to invoke a tool and invoke it for you and provide you the response.

#### Using Tool Responses

Answer the user's question using the response from the tool. Feel free to make it conversational. 
```


## Building a Weather Assistant from the Bottom-Up

- For example, suppose we wanted GPT to give us information about the current weather? How can we do that?
- Let's take a look at a sketch of a prompt

In [93]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
import openai


# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

In [94]:
prompt = """
You are a helpful AI weather assistant named Jarvis.

Your knowledge cut-off is: September 2021
Today's date: October 18, 2023

## Weather Data

#### Edinburg, TX

- Time: '2023-10-18T11:30'.
- Temperature: 78°F.
- Daytime: True
- Precipitation ('Rain'): 0.0
- Precipitation ('Showers'): 0.0
- Precipitation ('Snowfall'): 0.0

## Using Weather Data

Answer the user's question using the available weather data. Feel free to make it conversational.
Do not answer about cities you do not have weather data for.
"""

In [96]:
user_message = "What is the weather like in Edinburg, TX right now?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

The current weather in Edinburg, TX is 78°F with clear skies. It's a beautiful day to enjoy outdoor activities. Is there anything else you would like to know?


In [49]:
user_message = "What is the weather like in Brownsville, TX right now?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

I'm sorry, but I don't have the current weather data for Brownsville, TX. However, I can provide you with the weather information for Edinburg, TX, which is nearby. In Edinburg, TX, the temperature is currently 78°F, and it is daytime with no precipitation. If you have any other questions or need assistance with anything else, feel free to ask!


#### But how to automate?

- We have the basic sketch of a prompt but how do we dynamically insert weather data?
- Let's use an API!

#### Open-Meteo API

- Geocoding Endpoint: 
    * `GET https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json`
    * Search by city name, returns latitude longitude information for top 10 matches
- Weather Endpoint: 
    * `GET https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago`
    * Search by latitude and longitude, returns weather information

In [97]:
import requests

def geocode(city_name):
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response

In [98]:
def weather(latitude, longitude):
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response

In [99]:
print(geocode("Edinburg"))

{'results': [{'id': 4688275, 'name': 'Edinburg', 'latitude': 26.30174, 'longitude': -98.16334, 'elevation': 30.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4736286, 'admin2_id': 4697444, 'timezone': 'America/Chicago', 'population': 84497, 'postcodes': ['78539', '78540'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Texas', 'admin2': 'Hidalgo'}, {'id': 4257043, 'name': 'Edinburgh', 'latitude': 39.35422, 'longitude': -85.96666, 'elevation': 205.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 4921868, 'admin2_id': 4259727, 'admin3_id': 4254728, 'timezone': 'America/Indiana/Indianapolis', 'population': 4546, 'postcodes': ['46124'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Indiana', 'admin2': 'Johnson', 'admin3': 'Blue River Township'}, {'id': 4757240, 'name': 'Edinburg', 'latitude': 38.82095, 'longitude': -78.56585, 'elevation': 246.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 6254928, 'admin2_id': 4785252

In [53]:
print(weather(26.30174, -98.16334))

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.028967857360839844, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2023-10-18T11:45', 'interval': 900, 'temperature_2m': 26.5, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [107]:
prompt = """
You are a helpful AI weather assistant named Jarvis.

Your knowledge cut-off is: September 2021
Today's date: October 18, 2023

## Weather Data

#### Edinburg, TX

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.028967857360839844, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2023-10-18T11:45', 'interval': 900, 'temperature_2m': 26.5, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}

## Using Weather Data

Answer the user's question using the available weather data. Feel free to make it conversational.
Do not answer about cities you do not have weather data for.
"""

In [102]:
user_message = "What is the weather like in Edinburg, TX right now?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

The current weather in Edinburg, TX is 26.5°C with no precipitation. It's a nice day to be outside! Is there anything else you would like to know?


#### The Big Idea (Tool Pattern)

- We have _functions_ that use code to interact with the outside world
- Our functions pull live information from APIs
- If we develop a secret code between us and GPT we can let GPT tell _you_ when it wants to invoke a function and with what parameters
- Then _we_ can go do it on behalf of GPT and supply the results
- If this works, we can then automate this kind of interaction

## Applying the Tool Pattern "Old School Method"

- Tools: Functions your teach the LLM to use via "prompting"
- Using Tools: Teach the LLM to output "Tool: Weather('Mcallen, TX')" when someone asks for weather information for "Mcallen, TX")
- Invoking Tools: You watch for this type of output, and when you see it you invoke the function for the LLM and give it back the results.
- Returning Results: "Tool Result: <result>" add this to the list of messages in the interaction, then generate a new response from the LLM so that it can try to answer based on that knowledge.

In [70]:
prompt = """
You are a helpful AI assistant named Jarvis.

Your knowledge cut-off is: September 2021
Today's date: October 16, 2023

## Tools

You have access to the following tools:
- geocode(city): Tool to lookup the latitude and longitide of a US city. Does not accept state in the parameter. Do not specify the state. Example: geocode("New York")

## Tool Rules

When the user asks a question that can be answered by using a tool, you MUST do so. Do not answer from your training data.

## Using Tools

To use a tool, reply with the following prefix "Tool: " then append the tool call (like a function call). 

Behind the scenes, your software will pickup that you want to invoke a tool and invoke it for you and provide you the response.

## Using Tool Responses

Answer the user's question using the response from the tool. Feel free to make it conversational. 
"""

# This should spit out the secret 'code' to run the tool
user_message = "What is the latitude and longitude of Edinburg, TX?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)

message_history.append({"role": "user", "content": user_message})
message_history.append({"role": "assistant", "content": response})

print(response)

Tool: geocode("Edinburg")


In [71]:
# Let's help GPT out by running it and giving it the results
result = geocode("Edinburg")

# append tool response to message history
message_history.append({"role": "assistant", "content": f"Tool: {result}"})

# regenerate new response
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

The latitude and longitude of Edinburg, TX are approximately 26.30174° N and -98.16334° W, respectively.


## Working with Tool Chains

- Suppose you have multiple tools
- You want GPT to be able to string them together
- How do you do this? Just tell it that it can chain the tools together.

In [73]:
prompt = """
You are a helpful AI weather assistant named Jarvis.

Your knowledge cut-off is: September 2021
Today's date: October 16, 2023

## Tools

You have access to the following tools:
- geocode(city): Tool to lookup the latitude and longitide of a US city. Does not accept state in the parameter. Do not specify the state. Example: geocode("New York")
- weather(lat, long): Tool to lookup weather for a specific latitude and longitude. Example: weather(52.52, 13.41)

## Tool Rules

When the user asks a question that can be answered by using a tool, you MUST do so. Do not answer from your training data.

## Using Tools

To use a tool, reply with the following prefix "Tool: " then append the tool call (like a function call). 

Behind the scenes, your software will pickup that you want to invoke a tool and invoke it for you and provide you the response.

## Using Tool Responses

Answer the user's question using the response from the tool. Feel free to make it conversational. 

## Chaining Tools

You are allowed to chain together multiple calls to tools before giving an answer, if needed.
"""

user_message = "What is the weather like in Edinburg, TX right now?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)

message_history.append({"role": "user", "content": user_message})
message_history.append({"role": "assistant", "content": response})

print(response)

Tool: geocode("Edinburg")


In [74]:
# Let's help GPT out by running it and giving it the results
result = geocode("Edinburg")

# append tool response to message history
message_history.append({"role": "assistant", "content": f"Tool Result: {result}"})

# regenerate response
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

Tool: weather(26.30174, -98.16334)


In [75]:
# Let's help GPT out by running it and giving it the results
result = weather(26.30174, -98.16334)

# append tool response to message history
message_history.append({"role": "assistant", "content": f"Tool Result: {result}"})

# regenerate response
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
print(response)

The current weather in Edinburg, TX is 27.4°C with no precipitation. It's a sunny day!


#### Automating the Tool Pattern

- We manually helped out GPT by invoking the function call when we saw it
- We need to write code that can do this automatically when it spots that a tool should be used
- From a string like "Tool: weather(26.30174, -98.16334)" we need to be able to:
    - Detect that a tool is required
    - Extract the name of the tool
    - Extract the parameters
    - Invoke the function

In [103]:
def needs_tool(response):
    return "Tool:" in response

# Example
print(needs_tool("Tool: weather(26.30174, -98.16334)"))
print(needs_tool("It is bright and sunny today!"))

True
False


In [104]:
import re

def extract_call(string):
    # regex pattern
    pattern = r'Tool: (\w+)\((.*?)\)'
    match = re.search(pattern, string)
    if match:
        tool_name = match.group(1)
        parameters = match.group(2).replace('"', '').split(', ')
        return tool_name, parameters
    else:
        return None, None
    
# Example
ai_reply = "Tool: weather(26.30174, -98.16334)"
if needs_tool(ai_reply):
    print(extract_call(ai_reply))

('weather', ['26.30174', '-98.16334'])


In [105]:
def invoke_tool(response):
    tool_name, parameters = extract_call(response)
    
    if tool_name == "geocode":
        tool_result = geocode(*parameters)
    elif tool_name == "weather":
        tool_result = weather(*parameters)
        
    return tool_result

# Example
ai_reply = "Tool: weather(26.30174, -98.16334)"
if needs_tool(ai_reply):
    tool_result = invoke_tool(ai_reply)
    print(f"Tool Result: {tool_result}")

Tool Result: {'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.031948089599609375, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2023-10-18T16:30', 'interval': 900, 'temperature_2m': 30.4, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [40]:
user_message = "What is the weather like in Edinburg, TX right now?"
message_history = []
response = get_ai_reply(user_message, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)

message_history.append({"role": "user", "content": user_message})

while(needs_tool(response)):
    print(response)
    message_history.append({"role": "assistant", "content": response})
    tool_result = invoke_tool(response)
    message_history.append({"role": "assistant", "content": f"Tool Result: {tool_result}"})
    response = get_ai_reply(None, model="gpt-3.5-turbo", system_message=prompt, message_history=message_history)
    
print(response)


Tool: geocode("Edinburg")
Tool: weather(26.30174, -98.16334)
The current weather in Edinburg, TX is 23.7°C with no precipitation. It is currently daytime.


## Putting it all together

In [106]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import gradio as gr
import openai
import requests
import re
# ---------------------------------------------------------------------------------------

def search(text):
    collection.query()
def geocode(city_name):
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
def weather(latitude, longitude):
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
# ---------------------------------------------------------------------------------------
def needs_tool(response):
    return "Tool:" in response

def extract_call(string):
    # regex pattern
    pattern = r'Tool: (\w+)\((.*?)\)'
    match = re.search(pattern, string)
    if match:
        tool_name = match.group(1)
        parameters = match.group(2).replace('"', '').split(', ')
        return tool_name, parameters
    else:
        return None, None
    
def invoke_tool(response):
    tool_name, parameters = extract_call(response)
    
    if tool_name == "search_movies":
        tool_result = search_movies(*parameters)
    elif tool_name == "weather":
        tool_result = weather(*parameters)
        
    return tool_result
# ---------------------------------------------------------------------------------------
# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()
# ---------------------------------------------------------------------------------------
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are a helpful AI weather assistant named Jarvis.

        Your knowledge cut-off is: September 2021
        Today's date: October 16, 2023

        ## Tools

        You have access to the following tools:
        - geocode(city): Tool to lookup the latitude and longitide of a US city. Does not accept state in the parameter. Do not specify the state. Example: geocode("New York")
        - weather(lat, long): Tool to lookup weather for a specific latitude and longitude. Example: weather(52.52, 13.41)
        - search_movies(query): Tool to lookup top 10 movies for a search query
        ## Tool Rules

        When the user asks a question that can be answered by using a tool, you MUST do so. Do not answer from your training data.

        ## Using Tools

        To use a tool, reply with the following prefix "Tool: " then append the tool call (like a function call). 

        Behind the scenes, your software will pickup that you want to invoke a tool and invoke it for you and provide you the response.

        ## Using Tool Responses

        Answer the user's question using the response from the tool. Feel free to make it conversational. 

        ## Chaining Tools

        You are allowed to chain together multiple calls to tools before giving an answer, if needed.
        """
        ai_reply = get_ai_reply(message, model="gpt-3.5-turbo", system_message=prompt.strip(), message_history=history_state)
            
        # Append the user's message and the AI's reply to the history_state list
        history_state.append({"role": "user", "content": message})
        history_state.append({"role": "assistant", "content": ai_reply})
        
        while(needs_tool(ai_reply)):
            tool_result = invoke_tool(ai_reply)
            history_state.append({"role": "assistant", "content": f"Tool Result: {tool_result}"})
            ai_reply = get_ai_reply(None, model="gpt-3.5-turbo", system_message=prompt.strip(), message_history=history_state)
            history_state.append({"role": "assistant", "content": ai_reply})
            
        # Append the user's message and the AI's reply to the chatbot_messages list for the UI
        chatbot_messages.append((message, ai_reply))

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return None, chatbot_messages, history_state

# Define a function to launch the chatbot interface using Gradio
def get_chatbot_app():
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        # Create a chatbot interface for the conversation
        chatbot = gr.Chatbot(label="Conversation")
        # Create a textbox for the user's message
        message = gr.Textbox(label="Message")
        # Create a state object to store the conversation history
        history_state = gr.State()
        # Create a button to send the user's message
        btn = gr.Button(value="Send")

        # Connect the send button to the chat function
        btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app
# ---------------------------------------------------------------------------------------        
# Call the launch_chatbot function to start the chatbot interface using Gradio
app = get_chatbot_app()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://6d65624d0d263e9f0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/openai/api_requestor.py", line 331, in handle_error_response
    error_data = resp["error"]
TypeError: string indices must be integers

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/folders/93/kmn7sjzd6gx6c0h6dgdt2src0000gn/T/ipykernel_38668/2549720379.py", line 127, in chat
    ai_reply = get_ai_reply(message, model="gpt-3.5-turbo", system_message=prompt.strip(), message_history=history_state)
  File "/var/folders/93/kmn7sjzd6gx6c0h6dgdt2src0000gn/T/ipykernel_38668/2549720379.py", line 80, in get_ai_reply
    completion = openai.ChatCompletion.create(
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/openai